In [1]:
import numpy as np
import cvxopt
import libsvm.svmutil as svm

In [2]:
TRAIN_DATA = "mnist/train.csv"
TEST_DATA = "mnist/test.csv"
K = 10
GAMMA = 0.05

In [3]:
def extract_data(file: str, subset: list[int]):
	data = np.genfromtxt(file, delimiter=',')
	X = data[:, :-1] / 255
	Y = np.array(data[:, -1], np.int32)
	indices = np.where(np.vectorize(lambda y: y in subset)(Y))
	X = X[indices]
	Y = Y[indices]
	Y = np.reshape(Y, (Y.shape[0], 1))
	return X, Y

In [4]:
def gaussian_prod(X1: np.ndarray, X2: np.ndarray, gamma: float):
	prod1 = np.reshape(np.einsum('ij,ij->i', X1, X1), (X1.shape[0], 1))
	prod2 = np.reshape(np.einsum('ij,ij->i', X2, X2), (X2.shape[0], 1))
	prod = prod1 + prod2.T - 2 * np.matmul(X1, X2.T)
	return np.exp(-gamma * prod)

In [5]:
def svm_cvxopt(X: np.ndarray, Y: np.ndarray, c: float, tol: float, find_prod, gamma: float):
	shape = (Y.shape[0], 1)

	prod = find_prod(X, X, gamma)
	P = cvxopt.matrix(np.matmul(Y, Y.T) * prod)

	q = cvxopt.matrix(-np.ones(shape))

	G = np.identity(shape[0])
	G = cvxopt.matrix(np.append(G, -G, axis=0))

	h = np.ones(shape)
	h = cvxopt.matrix(np.append(c * h, 0 * h, axis=0))

	A = cvxopt.matrix(Y.T, tc='d')

	b = cvxopt.matrix(0.0)
	
	sol = cvxopt.solvers.qp(P, q, G, h, A, b, options={'show_progress': False})
	if sol['status'] == "unknown":
		return None

	alpha = np.reshape(np.array(sol['x']), shape)
	inner_prod = np.sum(alpha * Y * prod, 0)

	indices = [i for i in range(shape[0]) if alpha[i] > tol]

	M = max(indices, key=lambda i: -float("inf") if Y[i] == 1 or alpha[i] >= c - tol else inner_prod[i])
	m = min(indices, key=lambda i: float("inf") if Y[i] == -1 or alpha[i] >= c - tol else inner_prod[i])
	b = -(inner_prod[M] + inner_prod[m]) / 2

	return indices, alpha[indices], b

In [6]:
def gaussian_prediction(alpha: np.ndarray, support_Y: np.ndarray, support_X: np.ndarray, b: float, X: np.ndarray):
	return np.reshape(np.where(np.sum(alpha * support_Y * gaussian_prod(support_X, X, GAMMA), 0) + b >= 0, 1, -1), (X.shape[0], 1))

In [7]:
def accuracy(X: np.ndarray, Y: np.ndarray, pred):
	return 100 * sum(pred(X) == Y)[0] / Y.shape[0]

In [8]:
X_train, Y_train = extract_data(TRAIN_DATA, range(K))
X_test, Y_test = extract_data(TEST_DATA, range(K))

In [9]:
def kC2_classifier_cvxopt(X: np.ndarray, Y: np.ndarray, c: float, tol: float, gamma: float, k: int):
	classes = [list(np.where(Y == i)[0]) for i in range(K)]
	classifier = dict()
	for i in range(k):
		for j in range(i + 1, k):
			X_ij, Y_ij = X[classes[i] + classes[j]], Y[classes[i] + classes[j]]
			Y_ij = np.where(Y_ij == j, 1, -1)
			classifier[i, j] = svm_cvxopt(X_ij, Y_ij, c, tol, gaussian_prod, gamma)
			indices = classifier[i, j][0]
			classifier[i, j] = X_ij[indices], Y_ij[indices], classifier[i, j][1], classifier[i, j][2]
	return classifier

In [10]:
def predict_k_cvxopt(classifier: dict, X: np.ndarray, k: int):
	votes = np.zeros((X.shape[0], k))
	for i in range(k):
		for j in range(i + 1, k):
			curr_classifier = classifier[i, j]
			preds = gaussian_prediction(curr_classifier[2], curr_classifier[1], curr_classifier[0], curr_classifier[3], X)
			for example, pred in zip(votes, preds):
				if pred == 1:
					example[j] += 1
				else:
					example[i] += 1
	return np.reshape(votes.argmax(1), (X.shape[0], 1))

In [ ]:
def accuracy_util_cvxopt(X_train: np.ndarray, Y_train: np.ndarray, X_test: np.ndarray, Y_test: np.ndarray, pred):
	train_accuracy = accuracy(X_train, Y_train, pred)
	test_accuracy = accuracy(X_test, Y_test, pred)
	return train_accuracy, test_accuracy

In [11]:
classifier = kC2_classifier_cvxopt(X_train, Y_train, 1.0, 1e-4, GAMMA, K)
accuracy_util_cvxopt(X_train, Y_train, X_test, Y_test, lambda X: predict_k_cvxopt(classifier, X, K))

97.24

In [12]:
def kC2_classifier_libsvm(X: np.ndarray, Y: np.ndarray, c: float, kernel, gamma: float):
	params = '-t {} -c {} -q'.format(kernel, c)
	if gamma:
		params += ' -g {}'.format(gamma)
	model = svm.svm_train(Y.T[0], X, params)
	indices = model.get_sv_indices()
	for i in range(len(indices)):
		indices[i] -= 1
	alpha = np.array(model.get_sv_coef(), ndmin=2).T
	return indices, alpha, model

In [13]:
def accuracy_util_libsvm(X_train: np.ndarray, Y_train: np.ndarray, X_test: np.ndarray, Y_test: np.ndarray, model):
	train_accuracy = svm.svm_predict(Y_train.T[0], X_train, model, '-q')[1][0]
	test_accuracy = svm.svm_predict(Y_test.T[0], X_test, model, '-q')[1][0]
	return train_accuracy, test_accuracy

In [14]:
indices, alpha, model = kC2_classifier_libsvm(X_train, Y_train, 1.0, svm.RBF, GAMMA)
print(accuracy_util_libsvm(X_train, Y_train, X_test, Y_test, model))

(99.92, 97.23)
